In [1]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.restaurants

In [2]:
coll.drop_indexes()

# Covered queries

In [3]:
coll.find_one()

{'_id': ObjectId('58dae60d8b8e4054555b72f8'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

In [4]:
coll.create_index([
    ('borough', 1),
    ('cuisine', 1),
    ('name', 1)
])

'borough_1_cuisine_1_name_1'

In [5]:
q = coll.find(
    {
        'borough': 'Bronx'
    },
    {
        'cuisine': 1,
        'name': 1
    }
)
list(q.limit(5))

[{'_id': ObjectId('58dae60d8b8e4054555b9c74'),
  'cuisine': 'African',
  'name': 'Accra Restaurant'},
 {'_id': ObjectId('58dae60e8b8e4054555baa96'),
  'cuisine': 'African',
  'name': 'Accra Restaurant'},
 {'_id': ObjectId('58dae60e8b8e4054555bb05e'),
  'cuisine': 'African',
  'name': 'Adaya African American Restaurant'},
 {'_id': ObjectId('58dae60e8b8e4054555bc6b7'),
  'cuisine': 'African',
  'name': 'African Last Stop'},
 {'_id': ObjectId('58dae60d8b8e4054555b822d'),
  'cuisine': 'African',
  'name': 'African Maquis La Plantation'}]

In [6]:
from pprint import pprint
def plan_summary(plan):
    winningPlan = plan['queryPlanner']['winningPlan']
    stages = []
    stage = plan['executionStats']['executionStages']
    while stage:
        inputStage = stage.pop('inputStage', None)
        stages.append(stage)
        stage = inputStage
    stages.reverse()
    print('Execution Stages')
    for stage in stages:
        pprint(stage)
        print('--')

In [7]:
plan_summary(q.explain())

Execution Stages
{'advanced': 5,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'borough': ['["Bronx", "Bronx"]'],
                 'cuisine': ['[MinKey, MaxKey]'],
                 'name': ['[MinKey, MaxKey]']},
 'indexName': 'borough_1_cuisine_1_name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 0,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'borough': 1, 'cuisine': 1, 'name': 1},
 'keysExamined': 5,
 'multiKeyPaths': {'borough': [], 'cuisine': [], 'name': []},
 'nReturned': 5,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 5}
--
{'advanced': 5,
 'alreadyHasObj': 0,
 'docsExamined': 5,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 0,
 'nReturned': 5,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'stage': 'FETCH',
 'works': 5}
-

In [8]:
q = coll.find(
    {
        'borough': 'Bronx'
    },
    {
        'cuisine': 1,
        'name': 1,
        '_id': 0
    }
)
list(q.limit(5))

[{'cuisine': 'African', 'name': 'Accra Restaurant'},
 {'cuisine': 'African', 'name': 'Accra Restaurant'},
 {'cuisine': 'African', 'name': 'Adaya African American Restaurant'},
 {'cuisine': 'African', 'name': 'African Last Stop'},
 {'cuisine': 'African', 'name': 'African Maquis La Plantation'}]

In [9]:
plan_summary(q.explain())

Execution Stages
{'advanced': 5,
 'direction': 'forward',
 'dupsDropped': 0,
 'dupsTested': 0,
 'executionTimeMillisEstimate': 0,
 'indexBounds': {'borough': ['["Bronx", "Bronx"]'],
                 'cuisine': ['[MinKey, MaxKey]'],
                 'name': ['[MinKey, MaxKey]']},
 'indexName': 'borough_1_cuisine_1_name_1',
 'indexVersion': 1,
 'invalidates': 0,
 'isEOF': 0,
 'isMultiKey': False,
 'isPartial': False,
 'isSparse': False,
 'isUnique': False,
 'keyPattern': {'borough': 1, 'cuisine': 1, 'name': 1},
 'keysExamined': 5,
 'multiKeyPaths': {'borough': [], 'cuisine': [], 'name': []},
 'nReturned': 5,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'seeks': 1,
 'seenInvalidated': 0,
 'stage': 'IXSCAN',
 'works': 5}
--
{'advanced': 5,
 'executionTimeMillisEstimate': 0,
 'invalidates': 0,
 'isEOF': 0,
 'nReturned': 5,
 'needTime': 0,
 'needYield': 0,
 'restoreState': 0,
 'saveState': 0,
 'stage': 'PROJECTION',
 'transformBy': {'_id': 0, 'cuisine': 1, 'name': 1

# Geospatial Indexing

In [10]:
coll.create_index([
    ('address.coord', '2dsphere')  # longitude, latitude 
])

'address.coord_2dsphere'

In [11]:
doc = coll.find_one()

In [12]:
doc

{'_id': ObjectId('58dae60d8b8e4054555b72f8'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

In [13]:
q = coll.find(
    {'address.coord': {
        '$nearSphere': doc['address']['coord']
    }},
    {'_id': 0, 'name': 1, 'address.coord': 1}
).limit(5)
list(q)

[{'address': {'coord': [-73.856077, 40.848447]},
  'name': 'Morris Park Bake Shop'},
 {'address': {'coord': [-73.8560691, 40.8483622]}, 'name': "Luciano'S Pizza"},
 {'address': {'coord': [-73.8559476, 40.8479438]},
  'name': 'La Masa Restaurant'},
 {'address': {'coord': [-73.85517089999999, 40.8483418]},
  'name': 'Doyles Pub'},
 {'address': {'coord': [-73.856526, 40.847648]}, 'name': "Patsy'S Pizzeria"}]

In [14]:
point = {
    '$type': 'Point',
    'coordinates': doc['address']['coord']
}
q = coll.find(
    {'address.coord': {'$nearSphere': {'$geometry': point}}},
    {'_id': 0, 'name': 1, 'address.coord': 1}
).limit(5)
list(q)

[{'address': {'coord': [-73.856077, 40.848447]},
  'name': 'Morris Park Bake Shop'},
 {'address': {'coord': [-73.8560691, 40.8483622]}, 'name': "Luciano'S Pizza"},
 {'address': {'coord': [-73.8559476, 40.8479438]},
  'name': 'La Masa Restaurant'},
 {'address': {'coord': [-73.85517089999999, 40.8483418]},
  'name': 'Doyles Pub'},
 {'address': {'coord': [-73.856526, 40.847648]}, 'name': "Patsy'S Pizzeria"}]

In [15]:
res = db.command('geoNear', 'restaurants', near=point, spherical=True)
res

{'ok': 1.0,
 'results': [{'dis': 0.0,
   'obj': {'_id': ObjectId('58dae60d8b8e4054555b72f8'),
    'address': {'building': '1007',
     'coord': [-73.856077, 40.848447],
     'street': 'Morris Park Ave',
     'zipcode': '10462'},
    'borough': 'Bronx',
    'cuisine': 'Bakery',
    'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
      'grade': 'A',
      'score': 2},
     {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
     {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
     {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
     {'date': datetime.datetime(2011, 3, 10, 0, 0),
      'grade': 'B',
      'score': 14}],
    'name': 'Morris Park Bake Shop',
    'restaurant_id': '30075445'}},
  {'dis': 9.46324857371126,
   'obj': {'_id': ObjectId('58dae60e8b8e4054555ba4ae'),
    'address': {'building': '1005A',
     'coord': [-73.8560691, 40.8483622],
     'street': 'Morris Park Avenue',
     'zipcode': '10

In [16]:
for i, doc in enumerate(res['results']):
    if i > 5:
        break
    print('{:.1f} meters away: {}'.format(doc['dis'], doc['obj']['name']))

0.0 meters away: Morris Park Bake Shop
9.5 meters away: Luciano'S Pizza
57.1 meters away: La Masa Restaurant
77.2 meters away: Doyles Pub
96.6 meters away: Patsy'S Pizzeria
101.9 meters away: New Fresco Toetillas Tommy'S Kitchen Inc


# Full-text search and indexing

In [17]:
import re
q_re = coll.find({'name': re.compile('^Dunkin')})
q_re.count()

547

In [18]:
coll.create_index([('name', 'text')])

'name_text'

In [19]:
q_text = coll.find({'$text': {
    '$search': 'Dunkin',
    '$caseSensitive': True
}})
q_text.count()

553

In [20]:
found_with_re = {doc['_id'] for doc in q_re}
found_with_text = {doc['_id'] for doc in q_text}

In [21]:
print('Found with re but not text:')
for _id in found_with_re - found_with_text:
    print(coll.find_one({'_id': _id}, {'name': 1}))
print('Found with text but not re:')
for _id in found_with_text - found_with_re:
    print(coll.find_one({'_id': _id}, {'name': 1}))

Found with re but not text:
Found with text but not re:
{'_id': ObjectId('58dae60d8b8e4054555b9910'), 'name': "Taco Bell, Dunkin' Donuts"}
{'_id': ObjectId('58dae60d8b8e4054555b86c1'), 'name': "Baskin Robbins/Dunkin' Donuts"}
{'_id': ObjectId('58dae60e8b8e4054555baad4'), 'name': "St. John'S University-Dunkin Donuts"}
{'_id': ObjectId('58dae60e8b8e4054555bb8bd'), 'name': 'Popeyes/Dunkin Donuts'}
{'_id': ObjectId('58dae60e8b8e4054555bce17'), 'name': 'Hudson/Dunkin Donuts/Basking Robins'}
{'_id': ObjectId('58dae60e8b8e4054555bb05d'), 'name': 'Hudson News/Dunkin Donuts'}


In [22]:
coll.drop_indexes()

In [23]:
coll.create_index([("$**", "text")])

'$**_text'

In [24]:
q_text = coll.find({'$text': {
    '$search': 'donut'     # finds everything with donut in name, cuisine, borough, or address
}})
q_text.count()

595

In [25]:
q_text = coll.find({'$text': {
    '$search': 'queen'     # finds everything with queen in name, cuisine, borough, or address (incl. "Queens")
}})
q_text.count()

5673